# 使用textrank来求得关键句
## 一、TextRank算法介绍
### 1.1 PageRank算法
PageRank是根据网页之间相互的超链接计算网页重要性 的技术，是Google的创始人拉里·佩奇和谢尔盖·布林于1998年在斯坦福大学发明了这项技术。

提到重要性 我们就会联想到我们上一个实验也计算了每一个单词的重要性，但是在PageRank算法中是为每个页面计算重要性 ，而TextRank对PageRank算法做了改进，使其可以计算每一个句子的重要性 。
我们通过一个例子来理解PageRank：
![TextRank](./pagerank.png)

我们可以看到这个图中有不同颜色的球，代表的是不同的页面，而图中的箭头代表的是 超链接 。如图中D球指向A球，代表的就是D界面有指向A界面的超链接，相当于是D界面“推荐”了A界面。PageRank就是基于“推荐”的算法。

我们看到各个界面，既有推荐出去 (Out) 的箭头，也有别的界面推荐进来 (In) 的箭头。于是我们就可以列出一张邻接矩阵来描述整张图(忽略紫色的球！)。

0代表没有推荐，1代表的是有推荐。

每一列代表的是你推荐的页面 (Out)。如A列，代表的就是A推荐的页面。如表格所描述的，A谁也没有推荐。

每一行代表的是谁推荐了你 (In)。如A行，代表的就是推荐A的页面。如表格所描述的，D推荐了A。

in\out | A | B | C | D | E | F |
-------|---|---|---|---|---|---|
A|0|0|0|1|0|0|
B|0|0|1|1|1|1|
C|0|1|0|0|0|0|
D|0|0|0|0|1|0|
E|0|0|0|0|0|1|
F|0|0|0|0|1|0|

有了这张邻接矩阵作为基础，我们就能够计算PageRank的核心：每一个页面的分数S。
![TextRank](./equation.png)
Vi 代表的是每一个顶点，及页面。

d 代表的是一个阻尼常数 (0<d<1)，用来确保每一个页面都至少有 (1-d)的分数。

In(Vi) 代表的是推荐Vi的页面。

Out(Vi) 代表的是Vi推荐的页面。

|Out(Vi)| 代表的是Vi推荐页面的数量。

我们会发现，每一个页面的分数 S( Vi ) 都是依赖于别的页面的分数 S( Vj ) 的。我们需要对每个页面的分数进行初始化。初始化的数字并不重要，因为通过数学可以证明初始化的值对于最终的结果并不影响，只不过会影响算法迭代的次数。我们这里为了计算方便，把初始的 S 都设为 1 。 d 设为 0.85 (参考 S. Brin, L Page 1998年的论文)

就这样一步一步算出每一个顶点的值，然后接着从A页面开始继续算，一遍一遍迭代，直到每个页面的分数都不再变化为止。这样，我们就得到了每一个页面的评分 S

------------------------------------------------------------------------------

### 1.2 TextRank算法
当我们把PageRank应用到我们的文本当中去的时候，我们首先会发现的问题是，句子和句子之间 如何相互“推荐”?
TextRank给出了相应的算法。
![TextRank](./textrank.png)

    注意！这里 S 代表的不是PageRank中的分数，它代表的是 Sentence 代表的是每一个句子。

    Si 代表的是第 i 个句子。

    wk 代表的是句子中第 k 个单词。

    |Si| 代表的是句子中单词的个数。

    { wk| wk ∈ Si & wk ∈ Sj } 代表着同时在 Si 和 Sj 中出现的单词。

根据这个就可以求出两个句子之间的相似度，也就是推荐程度。

举个例子。

    I am a fool. A big fool. I like fool.

    前两句句子当中都有的单词是 a 和 fool，两个单词。

    第一句话和第三句话都有的单词是 I 和 fool。

    后两句句子当中都有的单词是 fool .

    第一句话长度是 4，第二句话长度是3, 第三句话长度是3.
    

----------------------------------------------------

Similarity(S1,S2) = 2 / ( log(3) + log(4) ) = 0.80

Similarity(S1,S3) = 2 / ( log(3) + log(3) ) = 0.91

Similarity(S2,S3) = 1 / ( log(3) + log(4) ) = 0.40

注意！两个句子的相似度没有指向性. 也就是说Similarity(S1,S2) = Similarity(S2,S1)
相似度不再只有0 和 1 ，它是浮点数的集合。

相似度 | 第一句 | 第二句 | 第三句 |
------|-------|-------|-------|
第一句|none|0.80|0.91|
第二句|0.80|none|0.40|
第三句|0.91|0.40|none|

我们写出来类似PageRank中的邻接矩阵。
在此基础上，我们可以应用新的PageRank算法来完成分数的计算
![TextRank](./textequation.png)

WS(Vi) 代表的是Vi这个页面的分数

d 代表的是一个阻尼常数 (0<d<1)，用来确保每一个页面都至少有 (1-d)的分数。

In(Vi) 代表的是推荐Vi的页面。

Out(Vi) 代表的是Vi推荐的页面。

wji 代表的是 Vi 和 Vj 之间的相似度。

我们拿第一句句子作为例子，看一看它的得分。同样的，初始分数都是 1 。

WS(V1) = (1 - 0.85) + 0.85 * 
( /*第二句句子*/ (0.80 * 1) / (0.80 + 0.40) + 
  /*第三句句子*/ (0.91 * 1) / (0.91 + 0.40) ) = 1.30
  
至此我们进一步迭代，计算出第二句和第三句的 分数 ，然后继续从第一句开始计算。直到最终每一句的分数不再变化为止。

## 二、实验步骤
### 2.1准备工作

In [14]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import math
from itertools import product,count
from string import punctuation
from heapq import nlargest

In [15]:
stopwords = set(stopwords.words('english')+list(punctuation))

### 2.2、计算相似性
这个函数计算两个句子之间的相似性。有关相似性的知识，请看上文的算法介绍。

In [16]:
'''
传入两个句子，
返回这两个句子相似度。
'''
def calculate_similarity(sen1,sen2):
    # 设置counter计数器
    counter = 0
    for word in sen1:
        if word in sen2:
            counter += 1
    return counter/(math.log(len(sen1))+math.log(len(sen2)))

### 2.3创建相似度邻接矩阵

In [17]:
"""
传入句子的列表
返回各个句子之间相似度的图
（邻接矩阵表示）
"""
def create_graph(word_sent):
    num = len(word_sent)
    # 初始化表
    board = [[0.0 for _ in range(num)] for _ in range(num)]
    
    for i,j in product(range(num),repeat=2):
        if i != j:
            board[i][j] = calculate_similarity(word_sent[i],word_sent[j])
    return board

### 2.4、根据PAGERANK算法，算出句子分数

In [21]:
"""
输入相似度邻接矩阵
返回各个句子的分数
"""
def weighted_pagerank(weight_graph):
    # 把初始化的分数值设置为0.5
    scores = [0.5 for _ in range(len(weight_graph))]
    old_scores = [0.0 for _ in range(len(weight_graph))]
    
    # 开始迭代
    while different(scores,old_scores):
        for i in range(len(weight_graph)):
            old_scores[i] = scores[i]
        for i in range(len(weight_graph)):
            scores[i] = calculate_score(weight_graph,scores,i)
    return scores

def different(scores,old_scores):
    flag = False
    for i in range(len(scores)):
        if math.fabs(scores[i]-old_scores[i]) >= 0.0001:
            flag = True
            break
    return flag

'''
根据公式求出指定句子的分数
'''
def calculate_score(weight_graph,scores,i):
    length = len(weight_graph)
    d = 0.85
    added_score = 0.0
    
    for j in range(length):
        fraction = 0.0
        denominator = 0.0
        # 先计算分子
        fraction = weight_graph[j][i] * scores[j]
        # 计算分母
        for k in range(length):
            denominator += weight_graph[j][k]
        added_score += fraction/denominator
    # 算出最终的分数
    weighted_scores = (1-d)+d*added_score
    
    return weighted_scores

### 3.5、找出分数最高的两个句子

In [22]:
def summarize(text,n):
    # 首先分出句子
    sents = sent_tokenize(text)
    # 然后分出单词
    # word_sent是一个二维的列表
    # word_sent[i]代表的是第i句
    # word_sent[i][j]代表的是
    # 第i句中的第j个单词
    word_sent = [word_tokenize(s.lower()) for s in sents]
    
    # 把停用词去除
    for i in range(len(word_sent)):
        for word in word_sent[i]:
            if word in stopwords:
                word_sent[i].remove(word)
    similarity_graph = create_graph(word_sent)
    scores = weighted_pagerank(similarity_graph)
    sent_selected = nlargest(n,zip(scores,count()))
    sent_index = []
    for i in range(n):
        sent_index.append(sent_selected[i][1])
    return [sents[i] for i in sent_index]

In [23]:
if __name__ == '__main__':
    with open('news.txt','r') as myfile:
        text = myfile.read().replace('\n','')
        print(summarize(text,2))

['The PHE website and app has a quiz that gives users a health score based on their lifestyle habits by asking questions such as, "Which snacks do you eat in a normal day?"', 'The campaign\'s clinical adviser, Prof Muir Gray, said it was about trying to make people have a different attitude to an "environmental problem".']
